In [54]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join
from tqdm import tqdm
import pandas as pd
import numpy as np
import joblib
import imodelsx.process_results
import sys
from sklearn.metrics import accuracy_score

sys.path.append('../experiments/')
results = []

results_dir = f'../results/distillation_cub/cub'
experiment_filename = '../experiments/distillation_cub.py'

# load the results in to a pandas dataframe
r = imodelsx.process_results.get_results_df(results_dir)
cols_varied = imodelsx.process_results.get_experiment_keys(
    r, experiment_filename)
print('experiment varied these params:', cols_varied)
r = imodelsx.process_results.fill_missing_args_with_default(
    r, experiment_filename)

# save results as csvs for backup
r.to_csv('../csv_results/distillation_cub.csv')

# get the breakdown of data in these groups
r.groupby(cols_varied).size()

100%|█████████████████████████████████████████| 20/20 [00:00<00:00, 1123.17it/s]

experiment varied these params: ['save_dir', 'teacher_path', 'max_rules', 'max_trees', 'max_depth', 'gpu']


save_dir                                                       teacher_path                                    max_rules  max_trees  max_depth  gpu
/home/mattyshen/DistillationEdit/results/distillation_cub/cub  cub/outputs/best_Joint0.01_big_model_1.pth      125        25         4          0      1
                                                                                                               150        30         4          0      1
                                                                                                               200        30         3          0      1
                                                                                                                          35         4          0      1
                                                               cub/outputs/best_Joint0.01_big_val_model_1.pth  125        25         4          0      1
                                                                                       

In [60]:
r.loc[~r.teacher_path.str.contains('val') & ~r.teacher_path.str.contains('2') & ~r.teacher_path.str.contains('3'), ~r.columns.str.contains('train')]

,save_dir,teacher_path,test_path,task_type,student_name,max_rules,max_trees,max_depth,min_impurity_decrease,metric,...,student_distillation_test_accuracy,student_prediction_test_accuracy,teacher_prediction_test_accuracy,depth,student_distillation_adap_interv_test_accuracy,student_prediction_adap_interv_test_accuracy,teacher_prediction_adap_interv_test_accuracy,student_distillation_rand_interv_test_accuracy,student_prediction_rand_interv_test_accuracy,teacher_prediction_rand_interv_test_accuracy
1,/home/mattyshen/DistillationEdit/results/disti...,cub/outputs/best_Joint0.01_model_1.pth,/home/mattyshen/ConceptBottleneck/CUB_processe...,regression,FIGSRegressor,125,25,4,0.10,accuracy,...,0.847774,0.748360,0.798412,3.0,0.839144,0.796168,0.837936,0.841905,0.762513,0.818605
4,/home/mattyshen/DistillationEdit/results/disti...,cub/outputs/best_Joint0.001_model_1.pth,/home/mattyshen/ConceptBottleneck/CUB_processe...,regression,FIGSRegressor,125,25,4,NaN,accuracy,...,0.702278,0.657232,0.831032,NaN,0.660338,0.628064,0.817570,0.674663,0.638592,0.817570
5,/home/mattyshen/DistillationEdit/results/disti...,cub/outputs/best_Joint0.01_model_1.pth,/home/mattyshen/ConceptBottleneck/CUB_processe...,regression,FIGSRegressor,125,25,4,NaN,accuracy,...,0.847774,0.748533,0.798412,NaN,0.838454,0.796168,0.838799,0.835002,0.759924,0.818433
7,/home/mattyshen/DistillationEdit/results/disti...,cub/outputs/best_Joint0.01_model_1.pth,/home/mattyshen/ConceptBottleneck/CUB_processe...,regression,FIGSRegressor,125,25,4,0.05,accuracy,...,0.847774,0.748533,0.798412,3.0,0.838454,0.796168,0.838799,0.836037,0.763203,0.821540
10,/home/mattyshen/DistillationEdit/results/disti...,cub/outputs/best_Joint0.01_big_model_1.pth,/home/mattyshen/ConceptBottleneck/CUB_processe...,regression,FIGSRegressor,125,25,4,NaN,accuracy,...,0.803590,0.702106,0.780117,NaN,0.768899,0.756127,0.811011,0.767173,0.713152,0.777528
15,/home/mattyshen/DistillationEdit/results/disti...,cub/outputs/best_Joint0.01_big_model_1.pth,/home/mattyshen/ConceptBottleneck/CUB_processe...,regression,FIGSRegressor,200,35,4,NaN,accuracy,...,0.833448,0.726959,0.780117,NaN,0.798757,0.777528,0.810321,0.803417,0.739386,0.777701
16,/home/mattyshen/DistillationEdit/results/disti...,cub/outputs/best_Joint0.01_big_model_1.pth,/home/mattyshen/ConceptBottleneck/CUB_processe...,regression,FIGSRegressor,200,30,3,NaN,accuracy,...,0.786331,0.688298,0.780117,NaN,0.752503,0.731101,0.806351,0.757680,0.695029,0.780980
19,/home/mattyshen/DistillationEdit/results/disti...,cub/outputs/best_Joint0.01_big_model_1.pth,/home/mattyshen/ConceptBottleneck/CUB_processe...,regression,FIGSRegressor,150,30,4,NaN,accuracy,...,0.815326,0.715223,0.780117,NaN,0.786158,0.766828,0.808595,0.785640,0.726096,0.777701


In [50]:
#.loc[r.teacher_path.str.contains('big') & ~r.teacher_path.str.contains('val'), :]

In [53]:
r[['teacher_path', 'num_trees', 'num_rules', 'depth', 'min_impurity_decrease', 'student_prediction_test_accuracy', 'student_prediction_adap_interv_test_accuracy','teacher_prediction_test_accuracy','teacher_prediction_adap_interv_test_accuracy']].sort_values('student_prediction_test_accuracy', ascending=False)

,teacher_path,num_trees,num_rules,depth,min_impurity_decrease,student_prediction_test_accuracy,student_prediction_adap_interv_test_accuracy,teacher_prediction_test_accuracy,teacher_prediction_adap_interv_test_accuracy
9,cub/outputs/best_Joint0.01_model_2.pth,22.0,125.0,3.0,0.05,0.758716,0.815671,0.804625,0.837245
11,cub/outputs/best_Joint0.01_model_2.pth,NaN,NaN,NaN,NaN,0.758716,0.815844,0.804625,0.836900
12,cub/outputs/best_Joint0.01_model_2.pth,22.0,125.0,3.0,0.10,0.758198,0.816362,0.804625,0.837591
5,cub/outputs/best_Joint0.01_model_1.pth,21.0,125.0,3.0,0.05,0.748533,0.796168,0.798412,0.838799
4,cub/outputs/best_Joint0.01_model_1.pth,NaN,NaN,NaN,NaN,0.748533,0.796168,0.798412,0.838799
0,cub/outputs/best_Joint0.01_model_1.pth,21.0,125.0,3.0,0.10,0.748360,0.796168,0.798412,0.837936
13,cub/outputs/best_Joint0.01_big_model_1.pth,NaN,NaN,NaN,NaN,0.726959,0.777528,0.780117,0.810321
17,cub/outputs/best_Joint0.01_big_model_1.pth,NaN,NaN,NaN,NaN,0.715223,0.766828,0.780117,0.808595
8,cub/outputs/best_Joint0.01_big_model_1.pth,NaN,NaN,NaN,NaN,0.702106,0.756127,0.780117,0.811011
16,cub/outputs/best_Joint0.01_model_3.pth,NaN,NaN,NaN,NaN,0.693131,0.740076,0.834657,0.849327


In [28]:
r.iloc[8,:]

save_dir                                           /home/mattyshen/DistillationEdit/results/disti...
teacher_path                                              cub/outputs/best_Joint0.01_big_model_1.pth
train_path                                         /home/mattyshen/ConceptBottleneck/CUB_processe...
test_path                                          /home/mattyshen/ConceptBottleneck/CUB_processe...
task_type                                                                                 regression
student_name                                                                           FIGSRegressor
max_rules                                                                                        200
max_trees                                                                                         35
max_depth                                                                                          4
metric                                                                                     

In [29]:
r.iloc[11,:]

save_dir                                           /home/mattyshen/DistillationEdit/results/disti...
teacher_path                                              cub/outputs/best_Joint0.01_big_model_1.pth
train_path                                         /home/mattyshen/ConceptBottleneck/CUB_processe...
test_path                                          /home/mattyshen/ConceptBottleneck/CUB_processe...
task_type                                                                                 regression
student_name                                                                           FIGSRegressor
max_rules                                                                                        150
max_trees                                                                                         30
max_depth                                                                                          4
metric                                                                                     

In [4]:
r.iloc[[1, 3], :] [['student_prediction_test_accuracy',
                    'teacher_prediction_test_accuracy',
       'student_prediction_adap_interv_test_accuracy',
       'teacher_prediction_adap_interv_test_accuracy',
       'student_prediction_rand_interv_test_accuracy',
       'teacher_prediction_rand_interv_test_accuracy']].mean()

student_prediction_test_accuracy                0.683120
teacher_prediction_test_accuracy                0.778478
student_prediction_adap_interv_test_accuracy    0.730583
teacher_prediction_adap_interv_test_accuracy    0.801433
student_prediction_rand_interv_test_accuracy    0.692354
teacher_prediction_rand_interv_test_accuracy    0.781584
dtype: float64

In [5]:
r.loc[r.teacher_path.str[22:26] == '0.01', :] [[
       'student_prediction_train_accuracy', 'student_prediction_test_accuracy',
       'teacher_prediction_train_accuracy', 'teacher_prediction_test_accuracy',
       'student_prediction_adap_interv_train_accuracy',
       'student_prediction_adap_interv_test_accuracy',
       'teacher_prediction_adap_interv_train_accuracy',
       'teacher_prediction_adap_interv_test_accuracy',
       'student_prediction_rand_interv_train_accuracy',
       'student_prediction_rand_interv_test_accuracy',
       'teacher_prediction_rand_interv_train_accuracy',
       'teacher_prediction_rand_interv_test_accuracy']].mean()

student_prediction_train_accuracy                0.840074
student_prediction_test_accuracy                 0.704522
teacher_prediction_train_accuracy                0.931568
teacher_prediction_test_accuracy                 0.799059
student_prediction_adap_interv_train_accuracy    0.855114
student_prediction_adap_interv_test_accuracy     0.754272
teacher_prediction_adap_interv_train_accuracy    0.930774
teacher_prediction_adap_interv_test_accuracy     0.822273
student_prediction_rand_interv_train_accuracy    0.842789
student_prediction_rand_interv_test_accuracy     0.714748
teacher_prediction_rand_interv_train_accuracy    0.922042
teacher_prediction_rand_interv_test_accuracy     0.804323
dtype: float64